In [39]:
import pandas as pd

In [40]:
data_health = pd.read_csv("../../Resources/Raw_data/health_data.csv")

In [41]:
needed_columns_h = ['Year', 'StateAbbr', 'StateDesc', 'Data_Value_Type', 'Data_Value', 
                  'TotalPopulation', 'Short_Question_Text']
raw_health = data_health[needed_columns_h]

In [42]:
for_filter = ['Obesity', 'Depression', 'Current Asthma']

In [43]:
filtered_health = raw_health[(raw_health['Short_Question_Text'].isin(for_filter)) & 
                             (raw_health['Data_Value_Type'] == 'Age-adjusted prevalence')]

In [44]:
grouped_health = filtered_health.groupby(['Year', 'StateAbbr', 'StateDesc', 
                                        'Short_Question_Text'])['Data_Value'].mean()
gr_health = grouped_health.reset_index()
gr_health.columns = ['Year', 'State Abbriviation', 'State', 'Health Condition', 'Condition Prevalence (%)']
final_health = gr_health[gr_health['State'] != 'United States']
final_health.head()

,Year,State Abbriviation,State,Health Condition,Condition Prevalence (%)
0,2021,AK,Alaska,Current Asthma,10.126667
1,2021,AK,Alaska,Depression,19.620000
2,2021,AK,Alaska,Obesity,35.166667
3,2021,AL,Alabama,Current Asthma,10.883582
4,2021,AL,Alabama,Depression,23.302985


In [45]:
data_aqi = pd.read_csv("../../Resources/Raw_data/aqi_2021.csv")

In [46]:
needed_columns_aqi = ['State', 'Median AQI']
raw_aqi = data_aqi[needed_columns_aqi]

In [47]:
grouped_aqi = raw_aqi.groupby(['State'])['Median AQI'].mean()
grouped_aqi_res = grouped_aqi.reset_index()
grouped_aqi_res.head()

,State,Median AQI
0,Alabama,35.466667
1,Alaska,17.250000
2,Arizona,48.769231
3,Arkansas,35.818182
4,California,47.754717


In [48]:
helth_aqi = pd.merge(final_health, grouped_aqi_res, on = 'State', how = 'inner')
helth_aqi.head()

,Year,State Abbriviation,State,Health Condition,Condition Prevalence (%),Median AQI
0,2021,AK,Alaska,Current Asthma,10.126667,17.250000
1,2021,AK,Alaska,Depression,19.620000,17.250000
2,2021,AK,Alaska,Obesity,35.166667,17.250000
3,2021,AL,Alabama,Current Asthma,10.883582,35.466667
4,2021,AL,Alabama,Depression,23.302985,35.466667


In [49]:
from geopy.geocoders import Nominatim
import time

In [50]:
states = helth_aqi['State'].unique()
state_coords = []

In [51]:
geolocator = Nominatim(user_agent="project_3_UofT", timeout=10)

for state in states:
    location = geolocator.geocode(state + ", USA")
    if location:
        state_coords.append({'State': state, 'Latitude': location.latitude, 'Longitude': location.longitude})
    else:
        state_coords.append({'State': state, 'Latitude': None, 'Longitude': None})
    time.sleep(1)

In [52]:
coordinates = pd.DataFrame(state_coords)
coordinates_unique = coordinates.drop_duplicates()

In [53]:
helth_aqi_geo = pd.merge(helth_aqi ,coordinates_unique, on = "State", how = 'inner')
helth_aqi_geo.head()

,Year,State Abbriviation,State,Health Condition,Condition Prevalence (%),Median AQI,Latitude,Longitude
0,2021,AK,Alaska,Current Asthma,10.126667,17.250000,64.445961,-149.680909
1,2021,AK,Alaska,Depression,19.620000,17.250000,64.445961,-149.680909
2,2021,AK,Alaska,Obesity,35.166667,17.250000,64.445961,-149.680909
3,2021,AL,Alabama,Current Asthma,10.883582,35.466667,33.258882,-86.829534
4,2021,AL,Alabama,Depression,23.302985,35.466667,33.258882,-86.829534


In [54]:
data_temp = pd.read_csv("../../Resources/Raw_data/temp.csv")

In [55]:
temp_2021 = data_temp[data_temp['year'] == 2021]

In [56]:
temp_2021_filtered = temp_2021[['state', 'average_temp']]
temp_2021_grouped = temp_2021_filtered.groupby(['state'])['average_temp'].mean()
temp_res = temp_2021_grouped.reset_index()
temp_res.columns = ['State', 'Average Temperature (F)']

In [57]:
helth_aqi_geo_temp = pd.merge(helth_aqi_geo, temp_res, on = 'State', how = "inner")
helth_aqi_geo_temp.head()

,Year,State Abbriviation,State,Health Condition,Condition Prevalence (%),Median AQI,Latitude,Longitude,Average Temperature (F)
0,2021,AL,Alabama,Current Asthma,10.883582,35.466667,33.258882,-86.829534,63.758333
1,2021,AL,Alabama,Depression,23.302985,35.466667,33.258882,-86.829534,63.758333
2,2021,AL,Alabama,Obesity,41.310448,35.466667,33.258882,-86.829534,63.758333
3,2021,AR,Arkansas,Current Asthma,10.177333,35.818182,35.204888,-92.447911,60.833333
4,2021,AR,Arkansas,Depression,26.244000,35.818182,35.204888,-92.447911,60.833333


In [58]:
from bs4 import BeautifulSoup
import requests

In [59]:
url = "https://www.currentresults.com/Weather/US/average-annual-state-sunshine.php"
while True :
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    if response.status_code == 200:
        break

In [60]:
soup

<!DOCTYPE html>

<html lang="en">
<head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<link href="/Css-RD/main-rd-jun11-18.css" rel="stylesheet"/>
<link href="/Css-RD/mediaq-rd-jun8-18.css" media="only screen and (min-width: 29em) " rel="stylesheet"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="/apple-touch-icon-180x180.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="/touch-icon-192x192.png" rel="icon" sizes="192x192"/>
<link href="/favicon.ico" rel="shortcut icon"/>
<meta content="#0A8BDF" name="theme-color"/>
<script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-1038710-1"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-1038710-1', { 'anonymize_ip': true });
</script>
<script async="" crossorigin="anonymous" src="https://pagead2.googlesyndication.com/pagead/js/adsbygoogle.js?client=ca-pub-8

In [61]:
tables = soup.find_all('table')
sunshine_tables = []

for table in tables:
    rows = table.find_all('tr')
    headers = [th.text for th in rows[0].find_all('th')]
    data = []
    for row in rows[1:]:
        cells = [td.text.strip() for td in row.find_all('td')]
        data.append(cells)
    df = pd.DataFrame(data, columns=headers)
    sunshine_tables.append(df)


In [62]:
combined_sunshine = pd.concat(sunshine_tables, ignore_index=True)

In [63]:
sunshine_filtered = combined_sunshine[['State', 'Clear Days']].copy()
sunshine_filtered['Clear Days'] = pd.to_numeric(sunshine_filtered['Clear Days'])
sunshine_filtered['% Clear Days'] = (sunshine_filtered['Clear Days'] / 365) * 100
sunshine_final = sunshine_filtered[['State', '% Clear Days']]
sunshine_final.head()

,State,% Clear Days
0,Alabama,27.123288
1,Alaska,16.712329
2,Arizona,52.876712
3,Arkansas,33.698630
4,California,40.000000


In [64]:
helth_aqi_geo_temp_sun = pd.merge(helth_aqi_geo_temp, sunshine_final, on = 'State', how = "inner")

helth_aqi_geo_temp_sun.head()

,Year,State Abbriviation,State,Health Condition,Condition Prevalence (%),Median AQI,Latitude,Longitude,Average Temperature (F),% Clear Days
0,2021,AL,Alabama,Current Asthma,10.883582,35.466667,33.258882,-86.829534,63.758333,27.123288
1,2021,AL,Alabama,Depression,23.302985,35.466667,33.258882,-86.829534,63.758333,27.123288
2,2021,AL,Alabama,Obesity,41.310448,35.466667,33.258882,-86.829534,63.758333,27.123288
3,2021,AR,Arkansas,Current Asthma,10.177333,35.818182,35.204888,-92.447911,60.833333,33.698630
4,2021,AR,Arkansas,Depression,26.244000,35.818182,35.204888,-92.447911,60.833333,33.698630


In [65]:
helth_aqi_geo_temp_sun['Condition Prevalence (%)'] = helth_aqi_geo_temp_sun['Condition Prevalence (%)'].round(2)
helth_aqi_geo_temp_sun['Average Temperature (F)'] = helth_aqi_geo_temp_sun['Average Temperature (F)'].round(2)
helth_aqi_geo_temp_sun['% Clear Days'] = helth_aqi_geo_temp_sun['% Clear Days'].round(2)
helth_aqi_geo_temp_sun['Median AQI'] = helth_aqi_geo_temp_sun['Median AQI'].round(2)
helth_aqi_geo_temp_sun = helth_aqi_geo_temp_sun.drop('Year', axis=1)
helth_aqi_geo_temp_sun.head()

,State Abbriviation,State,Health Condition,Condition Prevalence (%),Median AQI,Latitude,Longitude,Average Temperature (F),% Clear Days
0,AL,Alabama,Current Asthma,10.88,35.47,33.258882,-86.829534,63.76,27.12
1,AL,Alabama,Depression,23.30,35.47,33.258882,-86.829534,63.76,27.12
2,AL,Alabama,Obesity,41.31,35.47,33.258882,-86.829534,63.76,27.12
3,AR,Arkansas,Current Asthma,10.18,35.82,35.204888,-92.447911,60.83,33.70
4,AR,Arkansas,Depression,26.24,35.82,35.204888,-92.447911,60.83,33.70


In [66]:
helth_aqi_geo_temp_sun['ID'] = range(1, len(helth_aqi_geo_temp_sun) + 1)

helth_aqi_geo_temp_sun.head()

,State Abbriviation,State,Health Condition,Condition Prevalence (%),Median AQI,Latitude,Longitude,Average Temperature (F),% Clear Days,ID
0,AL,Alabama,Current Asthma,10.88,35.47,33.258882,-86.829534,63.76,27.12,1
1,AL,Alabama,Depression,23.30,35.47,33.258882,-86.829534,63.76,27.12,2
2,AL,Alabama,Obesity,41.31,35.47,33.258882,-86.829534,63.76,27.12,3
3,AR,Arkansas,Current Asthma,10.18,35.82,35.204888,-92.447911,60.83,33.70,4
4,AR,Arkansas,Depression,26.24,35.82,35.204888,-92.447911,60.83,33.70,5


In [69]:
population_path = '../../Resources/Raw_data/population.xlsx'
population_df = pd.read_excel(population_path)
subset_poulation = population_df.iloc[9:59, 0:2]
subset_poulation.columns = ['State', 'Population']
subset_poulation['State'] = subset_poulation['State'].str.\
    replace('.', '', regex=False).str.strip()

In [67]:
import sqlite3

In [68]:
with sqlite3.connect('../../Resources/Output/project3.sqlite') as con:
    helth_aqi_geo_temp_sun.to_sql('data', con=con, dtype={'ID': 'INTEGER PRIMARY KEY'}, if_exists= 'replace'
                                  , index=False)

In [70]:
with sqlite3.connect('../../Resources/Output/project3.sqlite') as con:
    subset_poulation.to_sql('population', con=con, dtype={'State': 'STRING PRIMARY KEY'}, if_exists= 'replace'
                                  , index=False)

: 